# Make your own forcing data from ERA5

Introduction: There are lots of forcing data can be used to drive the model, such as ERA5, NCEP (https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html), etc. Here, we recommend ERA5 to make your own single point forcing as the NCEP and others are too big to download. The ERA5 data can be obtained in a small scale, which usually light weight to download and store.

## Step 1: Download the required variables from the cdsapi

First, we will use the `cdsapi` package to download the data. If you don't have the package installed, you can install it using the following command:

```
pip install cdsapi

cat <<EOF > ~/.cdsapirc
url: {api-url}
key: {uid}:{api-key}
EOF 
```

[How to get your CDS API?](https://cds.climate.copernicus.eu/api-how-to)

We also can use the interface to download the data. The code below is an example of how to download the data using the interface.

ref: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

In [1]:
from pyclmuapp import era5_download, era5_to_forcing

In [2]:
# Download ERA5 data for a single month
# if file already exists, it will not be downloaded again
single = era5_download(year=2012, month=8, lat=51.5, lon=0.12, outputfolder='./data')
# it will take times to download the data
# including quequing time, and post-processing time, downloading time 
# is usually more than minutes
# here is the example, that we have already downloaded the data, so it will not download again

2024-06-19 16:53:22,363 INFO Welcome to the CDS
2024-06-19 16:53:22,364 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


download: ./data/era5_single_2012_08_51.5_0.12.nc


2024-06-19 16:53:22,512 INFO Request is completed
2024-06-19 16:53:22,513 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data2/adaptor.mars.internal-1718805274.044251-27464-9-e3bb1d14-cc8b-4e38-a6df-43b5e7fcb04e.nc to ./data/era5_single_2012_08_51.5_0.12.nc (123.8K)
2024-06-19 16:53:22,785 INFO Download rate 455.5K/s


In [3]:
lat, lon, zbot = 51.5, 0.12, 30
month, year = 8, 2012
# Convert ERA5 data to CLM forcing
forcing = era5_to_forcing(single=single, 
                          lat=lat, lon=lon, zbot=zbot, 
                          outputfile=f'./data/era5_forcing_{lat}_{lon}_{year}_{str(month).zfill(2)}.nc')

In [4]:
forcing

<xarray.Dataset> Size: 54kB
Dimensions:   (time: 744, y: 1, x: 1)
Coordinates:
  * time      (time) datetime64[ns] 6kB 2012-08-01 ... 2012-08-31T23:00:00
  * x         (x) int64 8B 1
  * y         (y) int64 8B 1
Data variables:
    Tair      (time, y, x) float64 6kB 287.3 286.9 286.9 ... 286.5 286.6 286.4
    PSurf     (time, y, x) float64 6kB 1.004e+05 1.004e+05 ... 1.019e+05
    Wind      (time, y, x) float64 6kB 4.518 4.587 4.362 ... 2.77 3.455 3.345
    Qair      (time, y, x) float64 6kB 0.009579 0.009393 ... 0.006647 0.006699
    Zbot      (time, y, x) float64 6kB 30.0 30.0 30.0 30.0 ... 30.0 30.0 30.0
    SWdown    (time, y, x) float64 6kB 1e-16 1e-16 1e-16 ... 1e-16 1e-16 1e-16
    LWdown    (time, y, x) float64 6kB 314.2 313.1 320.9 ... 348.0 347.9 355.8
    Prectmms  (time, y, x) float64 6kB 1e-16 1e-16 1e-16 ... 1e-16 1e-16 1e-16
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-19 13:54:45 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

### how to combine multiple months?

In [5]:
import xarray as xr
era5_list = []
# from 2002 to 2014
years = [2012]
# download data from January to December
months = range(1, 13)
lat, lon, zbot = 51.5, 0.12, 30
outputfile='./data/era5_forcing_{lat}_{lon}_{zbot}_{year}_{month}.nc'
for year in years:
    for month in months:
        single = era5_download(year=year, month=month, 
                                     lat=lat, lon=lon, outputfolder='./data')
        # Convert ERA5 data to CLM forcing
        forcing = era5_to_forcing(single=single, 
                                lat=lat, lon=lon, zbot=zbot,
                                outputfile=outputfile.format(lat=lat, lon=lon, 
                                                             zbot=zbot, year=year, 
                                                             month=str(month).zfill(2)))
        ds = xr.open_dataset(outputfile.format(lat=lat, lon=lon, 
                                               zbot=zbot, year=year, 
                                               month=str(month).zfill(2)))
        era5_list.append(ds)
era5 = xr.concat(era5_list, dim='time').sortby('time')
era5.to_netcdf(f'./data/era5_forcing_{lat}_{lon}_{zbot}.nc')

2024-06-19 16:53:23,352 INFO Welcome to the CDS
2024-06-19 16:53:23,353 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-19 16:53:23,431 INFO Request is completed
2024-06-19 16:53:23,431 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data5/adaptor.mars.internal-1718806471.8199303-11272-5-99a96606-9cc9-4e72-a494-52f46e6951a6.nc to ./data/era5_single_2012_01_51.5_0.12.nc (123.8K)


download: ./data/era5_single_2012_01_51.5_0.12.nc


2024-06-19 16:53:23,739 INFO Download rate 403.2K/s
2024-06-19 16:53:23,907 INFO Welcome to the CDS
2024-06-19 16:53:23,908 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-19 16:53:24,010 INFO Request is completed
2024-06-19 16:53:24,010 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data0/adaptor.mars.internal-1718806873.95887-2269-12-b1138185-65e1-483a-b4c5-34d7cc713a39.nc to ./data/era5_single_2012_02_51.5_0.12.nc (116K)


download: ./data/era5_single_2012_02_51.5_0.12.nc


2024-06-19 16:53:24,298 INFO Download rate 403.2K/s
2024-06-19 16:53:24,429 INFO Welcome to the CDS
2024-06-19 16:53:24,429 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-19 16:53:24,542 INFO Request is completed
2024-06-19 16:53:24,543 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data3/adaptor.mars.internal-1718807360.9600046-10821-10-67a81a24-5ef7-48e8-8b09-45b2e1e8035a.nc to ./data/era5_single_2012_03_51.5_0.12.nc (123.8K)


...

2024-06-19 16:53:28,053 INFO Download rate 349.1K/s
2024-06-19 16:53:28,188 INFO Welcome to the CDS
2024-06-19 16:53:28,188 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-19 16:53:28,351 INFO Request is completed
2024-06-19 16:53:28,352 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data7/adaptor.mars.internal-1718810003.660018-13547-11-e52b2cc4-0dd8-4635-9c94-a6137059dc6e.nc to ./data/era5_single_2012_11_51.5_0.12.nc (119.9K)


download: ./data/era5_single_2012_11_51.5_0.12.nc


2024-06-19 16:53:28,569 INFO Download rate 553.6K/s
2024-06-19 16:53:28,705 INFO Welcome to the CDS
2024-06-19 16:53:28,706 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


download: ./data/era5_single_2012_12_51.5_0.12.nc


2024-06-19 16:53:28,947 INFO Request is completed
2024-06-19 16:53:28,947 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data7/adaptor.mars.internal-1718810350.758121-9172-11-2f168d42-9bab-4c0d-bd3a-f564627b8548.nc to ./data/era5_single_2012_12_51.5_0.12.nc (123.8K)
2024-06-19 16:53:29,243 INFO Download rate 419.5K/s


In [6]:
era5

<xarray.Dataset> Size: 632kB
Dimensions:   (time: 8784, y: 1, x: 1)
Coordinates:
  * time      (time) datetime64[ns] 70kB 2012-01-01 ... 2012-12-31T23:00:00
  * x         (x) int64 8B 1
  * y         (y) int64 8B 1
Data variables:
    Tair      (time, y, x) float64 70kB 284.7 284.7 284.7 ... 282.6 282.2 282.1
    PSurf     (time, y, x) float64 70kB 9.985e+04 9.985e+04 ... 9.894e+04
    Wind      (time, y, x) float64 70kB 8.674 8.805 8.251 ... 6.712 6.847 6.578
    Qair      (time, y, x) float64 70kB 0.008033 0.0081 ... 0.006877 0.006826
    Zbot      (time, y, x) float64 70kB 30.0 30.0 30.0 30.0 ... 30.0 30.0 30.0
    SWdown    (time, y, x) float64 70kB 1e-16 1e-16 1e-16 ... 1e-16 1e-16 1e-16
    LWdown    (time, y, x) float64 70kB 364.3 364.9 366.4 ... 332.9 336.0 335.0
    Prectmms  (time, y, x) float64 70kB 8.402e-05 1.508e-05 ... 1.389e-06
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-19 14:14:43 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

### command line

In [7]:
! pyclmuapp --pyclmuapp_mode get_forcing \
    --lat 51.5 --lon 0.12 --zbot 30 \
    --start_year 2012 --end_year 2012 \
    --start_month 1 --end_month 2
# will download and save in the default folder `./era5_forcing/`
# the output file will be `./era5_forcing/era5_forcing_51.5_0.12_30_2012_01_2012_2.nc`

Namespace(init=False, pwd='/Users/user/Documents/GitHub/pyclmuapp', container_type='docker', input_path=None, output_path=None, log_path=None, scripts_path=None, pyclmuapp_mode='get_forcing', has_container=True, usr_domain=None, usr_forcing=None, usr_surfdata=None, output_prefix='_clm.nc', case_name='usp_case', run_startdate=None, stop_option='ndays', stop_n='1', run_type='coldstart', run_refcase='None', run_refdate='None', iflog=True, logfile='pyclmuapp.log', hist_type='GRID', hist_nhtfrq=1, hist_mfilt=1000000000, clean=False, surf_var=None, surf_action=0, forcing_var=None, forcing_action=0, script=None, urbsurf=None, soildata=None, pct_urban=[0, 0, 100.0], lat=51.5, lon=0.12, outputname='surfdata.nc', zbot=30, start_year=2012, end_year=2012, start_month=1, end_month=2)
download: ./era5_data/era5_single_2012_01_51.5_0.12.nc
2024-06-19 16:53:30,158 INFO Welcome to the CDS
2024-06-19 16:53:30,158 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-

**test the forcing data**

In [8]:
! pyclmuapp \
    --container_type docker \
    --iflog True \
    --logfile "pyclmuapp.log" \
    --usr_forcing "era5_data/era5_forcing_51.5_0.12_30_2012_1_2012_2.nc" \
    --usr_surfdata "inputfolder/surfdata.nc" \
    --RUN_STARTDATE "2012-01-01" --STOP_OPTION "ndays" --STOP_N "15" \
    --RUN_TYPE "coldstart" \
    --hist_type "GRID" --hist_nhtfrq "1" \
    --hist_mfilt "1000000000" \
    --output_prefix "_clm.nc" \
    --CASE_NAME "pyclmuapp" \
    --run_type "coldstart" --run_refcase "None" --run_refdate "None" --clean True

Namespace(init=False, pwd='/Users/user/Documents/GitHub/pyclmuapp', container_type='docker', input_path=None, output_path=None, log_path=None, scripts_path=None, pyclmuapp_mode='usp', has_container=True, usr_domain=None, usr_forcing='era5_data/era5_forcing_51.5_0.12_30_2012_1_2012_2.nc', usr_surfdata='inputfolder/surfdata.nc', output_prefix='_clm.nc', case_name='pyclmuapp', run_startdate='2012-01-01', stop_option='ndays', stop_n='15', run_type='coldstart', run_refcase='None', run_refdate='None', iflog=True, logfile='pyclmuapp.log', hist_type='GRID', hist_nhtfrq=1, hist_mfilt=1000000000, clean='True', surf_var=None, surf_action=0, forcing_var=None, forcing_action=0, script=None, urbsurf=None, soildata=None, pct_urban=[0, 0, 100.0], lat=None, lon=None, outputname='surfdata.nc', zbot=30, start_year=2012, end_year=2012, start_month=1, end_month=12)
Folder 'inputfolder' already exists.
Folder 'outputfolder' already exists.
Folder 'logfolder' already exists.
Folder 'scriptsfolder' already ex